### Steps to cluster query results

1. Convert table outputs (including column names into html like strings)
2. Use MarkupLM to generate feature vectors from these html strings
3. use DBSCAN to cluster the strings together

In [1]:
import os
os.chdir('./..')
import json
from collections import Counter
from transformers import AutoProcessor, MarkupLMModel
import sqlite3
import torch
from sklearn.cluster import DBSCAN
import numpy as np
from sklearn.preprocessing import StandardScaler

# loading in dev set
with open('./data/grid_search_subsampled_dev_set.json', 'r') as f:
    dev_set = json.load(f)

c:\Users\varun\AppData\Local\pypoetry\Cache\virtualenvs\sqlcopilot-cwclWRYE-py3.10\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# function to convert any sql query output - column_names: list[str] and result: list[tuple[str]] into a html string
def sql_result_to_html(
    column_names: list[str],
    result: list[tuple[str]],
    error: str = None
):

    # Generate HTML content
    html_content = """
<body>
    <table>
        <thead>
            <tr>
"""

    if error:
        html_content += f"                <th>{error}</th>\n"
        html_content += """
    </tr>
</thead>"""

    else:    
        # Add headers to the table
        for header in column_names:
            html_content += f"                <th>{header}</th>\n"

        html_content += """
            </tr>
        </thead>
        <tbody>
"""

        # Add rows for the data
        for row in result:
            html_content += "            <tr>\n"
            for cell in row:
                html_content += f"                <td>{cell}</td>\n"
            html_content += "            </tr>\n"

        # Close the HTML table and body
        html_content += """
        </tbody>
    </table>
</body>
"""
    return html_content

def html_to_features(
    html_string: str,
    markup_lm_processor: AutoProcessor,
    markup_lm_model: MarkupLMModel
):
    # Have to compromise with truncation, max context length of markup lm is 512
    encoding = markup_lm_processor(html_string, return_tensors="pt", truncation=True)

    outputs = markup_lm_model(**encoding)
    last_hidden_states = outputs.last_hidden_state.mean(dim=1)
    return last_hidden_states

def chunk_html(html_string: str, max_tokens: int, processor: AutoProcessor):
    tokens = processor(html_string, return_tensors="pt", truncation=False, padding=False)["input_ids"][0]
    chunks = []
    for i in range(0, len(tokens), max_tokens):
        chunks.append(html_string[i:i + max_tokens])
    return chunks

def chunked_html_to_features(
    html_string: str,
    markup_lm_processor: AutoProcessor,
    markup_lm_model: MarkupLMModel,
    max_tokens: int = 512
):
    chunks = chunk_html(html_string, max_tokens, markup_lm_processor)
    embeddings = []
    for chunk in chunks:
        # Have to compromise, max context length of markup lm is 512
        encoding = markup_lm_processor(chunk, return_tensors="pt", truncation=True, max_tokens=512)
        outputs = markup_lm_model(**encoding)
        embeddings.append(outputs.last_hidden_state.mean(dim=1))
    return torch.cat(embeddings, dim=0)

In [ ]:
import json

with open('./data/test.json', 'r') as f:
    queries = json.load(f)

queries = [(q, "california_schools") for q in (queries['equivalent_queries'] + queries['different_queries'])]

markup_processor = AutoProcessor.from_pretrained("microsoft/markuplm-base")
markup_model = MarkupLMModel.from_pretrained("microsoft/markuplm-base")

# executing the result of these queries and gathering them in a list
# for unsuccessful queries, we cluster them based on the semantic information present in the error messages
all_features = []
for query in queries:
    sql_query = query[0]
    db_id = query[1]
    try:
        # Connect to the SQLite database
        conn = sqlite3.connect(f"{os.environ['DB_ROOT_DIRECTORY']}/{db_id}/{db_id}.sqlite")
        cursor = conn.cursor()
        
        # Execute the query
        cursor.execute(sql_query)
        
        # Fetch all results
        results = cursor.fetchall()
        columns = [description[0] for description in cursor.description]
        html_result = sql_result_to_html(column_names=columns, result=results)
        # print(html_result)
        features = html_to_features(
            html_string=html_result, 
            markup_lm_processor=markup_processor, 
            markup_lm_model=markup_model,
        )
        all_features.append(features.detach().squeeze(dim=0))
        
            
    except sqlite3.Error as e:
        print(f"An error occurred: {e}")
        html_result = sql_result_to_html(error=e)
        features = html_to_features(
            html_string=html_result, 
            markup_lm_processor=markup_processor, 
            markup_lm_model=markup_model,
        )
        all_features.append(features.detach().squeeze(dim=0))

    finally:
        # Close the connection
        if conn:
            conn.close()

c:\Users\varun\AppData\Local\pypoetry\Cache\virtualenvs\sqlcopilot-cwclWRYE-py3.10\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


An error occurred: no such column: f.District
An error occurred: no such column: frpm.District
An error occurred: no such column: District
An error occurred: no such column: f.District
An error occurred: no such column: f.District
An error occurred: no such column: District
An error occurred: near "SELECT": syntax error
An error occurred: no such column: f.District
An error occurred: no such column: District
An error occurred: no such column: District
An error occurred: no such column: f.District
An error occurred: no such column: District
An error occurred: no such column: District
An error occurred: no such column: f.District
An error occurred: no such column: District
An error occurred: no such column: f.District
An error occurred: near "SELECT": syntax error
An error occurred: no such column: f.District
An error occurred: no such column: District
An error occurred: no such column: f.District
An error occurred: no such column: District
An error occurred: no such column: f.District
A

In [5]:
print(len(all_features))

4


In [6]:
print(all_features[0])

NameError: name 'all_features' is not defined

In [7]:
# clustering all_features
def cluster_sql_queries(embeddings: np.ndarray):
    """
    Cluster SQL query embeddings using DBSCAN.
    """
    # Normalize the embeddings
    scaler = StandardScaler()
    normalized_embeddings = scaler.fit_transform(embeddings)
    
    # Apply DBSCAN
    dbscan = DBSCAN(eps=0.5, min_samples=1)  # You may need to tune these parameters
    clusters = dbscan.fit_predict(normalized_embeddings)
    
    return clusters.tolist()

# test
clusters_DB = cluster_sql_queries(embeddings=np.array(all_features))

In [9]:
clusters_DB

[0, 1, 2, 3]

## Calculation of semantic entropy

In [10]:
# calculate semantic entropy for clusters
def calculate_semantic_entropy(clusters: list[int]) -> float:
    """
    Calculate the semantic entropy of the clusters using the formula: 
    -Sigma(Pi * log(Pi)), where Pi is the number of candidates in cluster i divided by all candidates.
    """
    cluster_counts = Counter(clusters)
    entropy = 0
    for cluster_id, count in cluster_counts.items():
        Pi = count / len(clusters)
        entropy -= Pi * np.log2(Pi)
    
    return entropy

In [11]:
print(calculate_semantic_entropy(clusters=clusters_DB))

2.0


In [12]:
# maximum possible value of entropy
np.log2(4)

2.0

In [19]:
from collections import defaultdict

# Sample data
clusters = [1, 1, 3, 2, 3, 2, 1, 3]
methods = ['a', 'b', 'a', 'c', 'b', 'a', 'c', 'b']

# Create a dictionary to store method counts for each cluster
cluster_method_counts = defaultdict(lambda: defaultdict(int))

# Populate the method counts for each cluster
for cluster, method in zip(clusters, methods):
    cluster_method_counts[cluster][method] += 1

# Compute percentages for each method in each cluster
cluster_percentages = {}
for cluster, method_counts in cluster_method_counts.items():
    total_count = sum(method_counts.values())
    cluster_percentages[cluster] = {
        method: (count / total_count) for method, count in method_counts.items()
    }

# Display the results
for cluster, percentages in sorted(cluster_percentages.items()):
    print(f"Cluster {cluster}:")
    for method, percentage in percentages.items():
        print(f"  Method {method}: {percentage:.2f}%")

Cluster 1:
  Method a: 0.33%
  Method b: 0.33%
  Method c: 0.33%
Cluster 2:
  Method c: 0.50%
  Method a: 0.50%
Cluster 3:
  Method a: 0.33%
  Method b: 0.67%


In [20]:
cluster_percentages

{1: {'a': 0.3333333333333333,
  'b': 0.3333333333333333,
  'c': 0.3333333333333333},
 3: {'a': 0.3333333333333333, 'b': 0.6666666666666666},
 2: {'c': 0.5, 'a': 0.5}}